In [49]:
#Ram Ram
#import networkx as nx
import random
from shapely.geometry import Point, Polygon
import numpy as np
import random
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np

In [50]:
max_X=200
max_Y=350
min_X=0
min_Y=0
step_length_for_divide_X=10
step_length_for_divide_Y=10
intersections=[]
geometric_centers=[]
for i in range(0,max_X+step_length_for_divide_X,step_length_for_divide_X):
    for j in range(0,max_Y+step_length_for_divide_Y,step_length_for_divide_Y):
        intersections.append(tuple((i,j)))
        #print((i,j))

In [51]:
xcoords=[]
ycoords=[]
for i in range(len(intersections)):
    xcoords.append(intersections[i][0])
    ycoords.append(intersections[i][1])
# plt.figure(figsize=(20,20))
# plt.scatter(xcoords,ycoords)
# print(intersections)

In [52]:
geometric_centers=[]
for item1 in intersections:
    for item2 in intersections:
        if item1 != item2:
            x1,y1=item1[0],item1[1]
            x2,y2=item2[0],item2[1]
            if abs(x2-x1)==step_length_for_divide_X and abs(y2-y1)==step_length_for_divide_Y:
                geometric_centers.append(tuple(((x1+x2)/2,(y1+y2)/2)))
geometric_centers=[*set(geometric_centers)]
# xcoords_centers=[]
# ycoords_centers=[]
# plt.figure(figsize=(20,20))
# for i in range(len(geometric_centers)):
#     xcoords_centers.append(geometric_centers[i][0])
#     ycoords_centers.append(geometric_centers[i][1])
# plt.scatter(xcoords,ycoords,color='blue')
# plt.scatter(xcoords_centers,ycoords_centers,color='red')
# print(geometric_centers)

In [53]:
#Without defining the power load of each area we are going to define the EV demand per fast charging demand point
# EV_demand={}
# min_EV_load=1
# max_EV_load=10
# for point in geometric_centers:
#     EV_demand[point]=random.randint(min_EV_load,max_EV_load)
#print(EV_demand)

In [54]:
#Defining EV Demand Via Text File
EV_demand={}
with open(r'C:\Users\Admin\Desktop\IPSO Charging Station Demand.txt') as f:
    text_charging_demand = f.read()
charging_values=text_charging_demand.split("\n")
for value in charging_values:
    v=value.split(" ")
    if "" in v:
        v.remove("")
    if "}" == v[2][-1]:
        v[2]=v[2][0:len(v[2])-1]
    dem=int(v[2].split(",")[0])
    origin=int(v[0].split("(")[1].split(",")[0].split(".")[0])
    dest=int(v[1].split(")")[0].split(".")[0])
    EV_demand[(origin,dest)]=dem

In [55]:
# plt.figure(figsize=(20,20))
# plt.scatter(xcoords_centers,ycoords_centers,color='red')
# for k,v in EV_demand.items():
#     plt.text(k[0],k[1],str(v),size=20)

In [56]:
#Formatting cell.....Nothing Imp
geometric_centers_list=[]
for point in geometric_centers:
    geometric_centers_list.append([float(point[0]),float(point[1])])

In [57]:
# nc=50
# charging_station_coordinates=[]
# for i in range(30):
#     x=random.randint(1,200)
#     y=random.randint(1,300)
#     charging_station_coordinates.append(tuple((x,y)))
# charging_station_coordinates_list=[]
# for point in charging_station_coordinates:
#     charging_station_coordinates_list.append([float(point[0]),float(point[1])])
# # charging_station_coordinates_list.append([float(0),float(0)])
# # charging_station_coordinates_list.append([float(max_X),float(0)])
# # charging_station_coordinates_list.append([float(0),float(max_Y)])
# # charging_station_coordinates_list.append([float(max_X),float(max_Y)])
# points = np.array(charging_station_coordinates_list)
# vor = Voronoi(points)
# fig = plt.figure()
# fig = voronoi_plot_2d(vor)
# fig.set_figwidth(20)
# fig.set_figheight(10)
# plt.show()

In [58]:
def region_to_vertex_mapper(regions,vertices):   
    region_to_vertex_mapping={}
    index=0
    new_regions=[]
    for region in regions:
        if len(region)!=0:
            new_regions.append(region)
    regions=new_regions.copy()
    for region in regions:
        region_to_vertex_mapping[index]=[]
        for vertex_index in region:
            if vertex_index!=-1 and vertex_index in range(0,len(vertices)):
                region_to_vertex_mapping[index].append(tuple((vertices[vertex_index][0],vertices[vertex_index][1])))
        if len(region_to_vertex_mapping)==0:
            del region_to_vertex_mapping[index]
            index=index
        else:
            index+=1    
    return region_to_vertex_mapping

In [59]:
#check if point is in triangle or not
def area(x1, y1, x2, y2, x3, y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1)+ x3 * (y1 - y2)) / 2.0)
def isInside(x1, y1, x2, y2, x3, y3, x, y):
    A = area (x1, y1, x2, y2, x3, y3)
    A1 = area (x, y, x2, y2, x3, y3)
    A2 = area (x1, y1, x, y, x3, y3)
    A3 = area (x1, y1, x2, y2, x, y)
    if(A == A1 + A2 + A3):
        return True
    else:
        return False

In [60]:
def get_point_belonging_dictionary(points,polygon_coordinates_dict):
    
    set_of_polygons={}
    index=0
    #print(polygon_coordinates_dict)
    for key in polygon_coordinates_dict.keys():
        #print(polygon_coordinate)
        if len(polygon_coordinates_dict[key])>3:
            set_of_polygons[index]=Polygon(polygon_coordinates_dict[key])
        else:
            set_of_polygons[index]=polygon_coordinates_dict[key]
        index+=1
    #print(set_of_polygons.keys(),polygon_coordinates_dict.keys())
    #print(set_of_polygons)
    point_region_dictionary={}
    for point in points:
        point_region_dictionary[tuple((point[0],point[1]))]=[]
    ctr=0
    for point in points:
        #print(point)
        index=0
        for key in set_of_polygons.keys():
            #print(np.array(polygon_coordinates_dict[key]))
            if len(np.array(polygon_coordinates_dict[key]))>3:
                p=Point(float(point[0]),float(point[1]))
                #print(set_of_polygons[key])
                p.within(set_of_polygons[key])
                if set_of_polygons[key].contains(p)==True:
                    ctr+=1
                    #print(set_of_polygons[key].contains(p))
                if set_of_polygons[key].contains(p):
                    point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
            elif len(np.array(polygon_coordinates_dict[key]))==3:
                if isInside(polygon_coordinates_dict[key][0][0],polygon_coordinates_dict[key][0][1],polygon_coordinates_dict[key][1][0],polygon_coordinates_dict[key][1][1],polygon_coordinates_dict[key][2][0],polygon_coordinates_dict[key][2][1],point[0],point[1]):
                    point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
#             elif len(np.array(polygon_coordinates_dict[key]))<3:
#                 point_region_dictionary[tuple((point[0],point[1]))]=[polygon_coordinates_dict[key],index]
            index+=1
    #print(ctr)
    return point_region_dictionary

#point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list,region_to_vertex_mapping)



In [61]:
def charging_station_to_region_no_mapper(vor):
    charging_station_to_region_no_mapping={}
    cutter=0
    for i in range(len(vor.regions)):
        if len(vor.regions[i])==0:
            cutter=i
            break
    for i in range(len(vor.points)):
        if vor.point_region[i]<=cutter:
            charging_station_to_region_no_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]
        else:
            charging_station_to_region_no_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]-1
    return charging_station_to_region_no_mapping
        
    

In [62]:
def get_nearest_charging_station(point,vor,charging_station_to_region_no_mapping):
    charging_station_to_vertex_mapping=charging_station_vertex_mapper(vor)
    min_dist_dict={}
    curr_dist=10000000000000.0
    curr_key=-1
    for key in charging_station_to_region_no_mapping.keys():
        temp=np.sqrt(((key[0]-point[0])**2)+((key[1]-point[1])**2))
        if temp<=curr_dist:
            curr_dist=temp
            curr_coord=key
    return charging_station_to_vertex_mapping[curr_coord]

In [63]:
def charging_station_vertex_mapper(vor):
    charging_station_to_vertex_mapping={}
    cutter=0
    for i in range(len(vor.regions)):
        if len(vor.regions[i])==0:
            cutter=i
            break
    for i in range(len(vor.points)):
        if vor.point_region[i]<=cutter:
            charging_station_to_vertex_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]
        else:
            charging_station_to_vertex_mapping[tuple((vor.points[i][0],vor.points[i][1]))]=vor.point_region[i]-1
    return charging_station_to_vertex_mapping

        
def region_no_to_charging_station_mapper(vor, charging_station_to_region_no_mapping, point_region_dictionary, region_to_vertex_mapping):
    region_no_to_charging_station_mapping={}
    for i in range(len(vor.regions)-1):
        for point in vor.points:
            if charging_station_to_region_no_mapping[tuple((point[0],point[1]))]==i:
                region_no_to_charging_station_mapping[i]=tuple((point[0],point[1]))

    for key in point_region_dictionary.keys():
        if len(point_region_dictionary[key])==0:
            region_no=get_nearest_charging_station(key,vor,charging_station_to_region_no_mapping)
            point_region_dictionary[key]=[region_to_vertex_mapping[region_no],region_no,region_no_to_charging_station_mapping[region_no]]
        else:
            point_region_dictionary[key]=[point_region_dictionary[key][0],point_region_dictionary[key][1],region_no_to_charging_station_mapping[point_region_dictionary[key][1]]]
    return region_no_to_charging_station_mapping

In [64]:
def get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list):
    total_demand={}
    for station in charging_station_coordinates_list:
        temp_demand=0
        
        for key in point_region_dictionary.keys():
            if point_region_dictionary[key][2]==tuple((station[0],station[1])) or point_region_dictionary[key][2]==tuple((station[1],station[0])):
                temp_demand+=EV_demand[key]
        total_demand[tuple((station[0],station[1]))]=temp_demand
    return total_demand
#total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)

    

In [65]:
def demand_point_to_station_mapper(point_region_dictionary):
    demand_to_charging_station_mapper={}
    for key in point_region_dictionary.keys():
        demand_to_charging_station_mapper[key]=point_region_dictionary[key][2]
    return demand_to_charging_station_mapper
#demand_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)

def station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list):
    station_to_demand_mapper_dict={}
    for station in charging_station_coordinates_list:
        station_to_demand_mapper_dict[tuple((station[0],station[1]))]=[]
    for key in point_region_dictionary.keys():
        try:
            station_to_demand_mapper_dict[point_region_dictionary[key][2]].append(key)
        except:
            station_to_demand_mapper_dict[tuple((point_region_dictionary[key][2][1],point_region_dictionary[key][2][0]))].append(key)
    return station_to_demand_mapper_dict
#station_to_demand_mapper_dict=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)


In [66]:
# The Split EV demand w.r.t service and Total EV demand has been verified
# elec_list=[]
# for key in point_region_dictionary.keys():
#     if point_region_dictionary[key][2] not in elec_list:
#         elec_list.append(point_region_dictionary[key][2])
# sum(total_demand.values()), sum(EV_demand.values())

##### No of Chargers in a Charging Station


$
N_i=\left[\frac{\mu_i \sum_{j \in J_{C N i}} n_j}{n_S}\right] i \in J_{C S}
$

$
$$\newline N_i \text{- No of Chargers in a Charging station i}$$
$$\newline \mu_i \text{- Probability of EV users arriving at a charging station i at the same time }$$
$$\newline n_s \text{- No of EVs in a queue that can be accepted by EV users}$$
$$\newline J_{CN_i} \text{- Collection of all fast charging demand points located in the service area i}$$
$$\newline n_j \text{- No of EV car demand in fast charging demand point j}$$
$

##### Get No of Chargers Function

In [67]:
def get_no_of_chargers( demand_by_service_area ,mu=0.6, max_users=3):
    no_of_chargers={}
    for k in demand_by_service_area.keys():
        no_of_chargers[k]=int((mu*demand_by_service_area[k])/max_users)
    #print(no_of_chargers)
    return no_of_chargers

### Objective Function and Constraints

$ \min F_{\cos t}=F_1+F_2+F_3 $
$ \newline F_1=\sum_{i \in J_{C S}}\left[f_{C S}\left(N_i\right) \frac{r_0\left(1+r_0\right)^z}{\left(1+r_0\right)^z-1}+u_{C S}\left(N_i\right)\right] $
$ \newline F_2=365 a p \sum_{i \in J_{C S}} \sum_{j \in J_{C N_i}} n_j \lambda_{i j} d_{i j} $

$ \newline f_{C S}\left(N_i\right)=W+q N_i+e N_i^2$
$ \newline N_{i \cdot \min } \leq N_i \leq N_{i \cdot \max } i \in J_{C S} $
$ \newline \lambda_{i j} d_{i j} \leq d_{\max } i \in J_{C S}, j \in J_{C N i} $
$ \newline \lambda_{i j} D_{i j} \geq D_{\min } i, j \in J_{C S} ; i \neq j $

### Where:

$ F_{cost} \text{- Annual Social cost of the charging station} $ 
$ \newline F_{1} \text{- Annual Construction and Operation Cost of the Charging Station} $
$ \newline F_{2} \text{- Annual Loss cost of EV users on the way to the Charging Station} $
$ \newline F_{3} \text{- Annual Network Loss Cost of the Powergrid} $
$ \newline J_{CS} \text{-Set of Charging Stations} $
$ \newline f_{CS}(N_i) \text{- Construction Cost Function of Charging Station i} $
$ \newline N_{i} \text{- No of Chargers in Charging Station i} $
$ \newline r_{0} \text{- Diccount Rate} $
$ \newline z \text{- Depreciation Rate} $
$ \newline u_{CS}{(N_i)} \text{- Operating Cost function of Charging Station i valued as certain percentage of construction cost} $
$ \newline W \text{-Investment cost which includes initial construction cost and land purchase cost } $
$ \newline q \text{- Price of Charger} $
$ \newline e \text{- Auxiliary Investment Coefficient} $
$ \newline a \text{- Amt of Electricity Consumed by EV to travel 1 km} $
$ \newline p \text{- Charging Price of EV} $
$ \newline J_{CN_{i}} \text{- Collection of Fast Charging Demand points in service area of charging station i} $
$ \newline \lambda_{ij} \text{- ZigZag Coefficient of Road between charging station i and fast charging demand point j} $
$ \newline d_{ij} \text{- Linear distance space between charging station i and fast charging demand point j} $
$ \newline T \text{- Daily Work Time of Charging Station} $
$ \newline M_i \text{- No. of Transformers in charging station i} $
$ \newline C_{Cu},C_{Fe} \text{- Copper and Iron Loss converted to Converted to Transformer} $
$ \newline C_L,C_W \text{- Line loss and Charging loss converted to Charger} $
$ \newline N_{i,min} \text{- Min no of Chargers in Charging Station i} $
$ \newline N_{i,max} \text{- Max no of Chargers in Charging Station i} $
$ \newline d_max \text{- Maximum Distance between charging station and fast charging demand point } $
$ \newline D_{ij} \text{- Linear Distance between charging stations i and j} $
$ \newline D_min \text{- Minimum Distance between charging stations} $

In [68]:
def calculate_F1(charging_station_coordinates_list,demand_by_service_area,r0=0.08, z=20, W=200*(10**4), q=5*(10**4), e=1.5*(10**4),mu=0.6, max_users=3,ni_min=10,ni_max=20,penalty_F1=1000):
    no_of_chargers=get_no_of_chargers( demand_by_service_area ,mu=0.6, max_users=3)
    fixed_value=0.0
    fixed_value=((r0)*((1+r0)**z))/(((1+r0)**z)-1)
    fcs={}
    total_value=0
    other_costs=0
    added_cost=0
    #print(fixed_value)
    for station in no_of_chargers.keys():
        fcs[station]=(W+(q*max(1,no_of_chargers[station]))+(e*((max(1,no_of_chargers[station]))**2)))
        fcs[station]+=(0.15*fcs[station])#ucs or operating cost
        #Doing Penalty for ni_min and ni_max corrections
        if no_of_chargers[station]<ni_min:
            other_costs+=abs((penalty_F1*(int(ni_min-no_of_chargers[station]))))
            fcs[station]+=abs((penalty_F1*(int(ni_min-no_of_chargers[station]))))
        elif no_of_chargers[station]>ni_max:
            other_costs+=abs((penalty_F1*(int(no_of_chargers[station]-ni_max))))
            fcs[station]+=abs((penalty_F1*(int(no_of_chargers[station]-ni_max)))) 
    #print(fcs)
    #print("***************************")
    #print(no_of_chargers,len(no_of_chargers.keys()))
    print("F1 OTHER COSTS=",other_costs)
    adder_text=""
    adder_text+="F1 OTHER COSTS="+str(other_costs)+"\n"
    return [sum(fcs.values()),adder_text]

        

In [69]:
def calculate_F2(point_region_dictionary,charging_station_coordinates_list,EV_demand,lambdaij=1.2,a=0.3,p=1,dmax=1.5,penalty_F2=100):
    f2={}
    other_costs=0
    station_to_demand_mapper_dict=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    for station in station_to_demand_mapper_dict.keys():
        f2[station]=0.0
        for demand_point in station_to_demand_mapper_dict[station]:
            dist=np.sqrt(((station[0]-demand_point[0])**2)+((station[1]-demand_point[1])**2))
            f2[station]+=(lambdaij*EV_demand[demand_point]*dist)
            if (dist*lambdaij) > dmax:
                f2[station]+=penalty_F2*((dist*lambdaij)-dmax)
                other_costs+=penalty_F2*((dist*lambdaij)-dmax)
    print("F2 OTHER COSTS=",other_costs)
    adder_text=""
    adder_text+="F2 OTHER COSTS="+str(other_costs)+"\n"
    return [365*a*p*sum(f2.values()),adder_text]

In [70]:
def calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,demand_by_service_area, EV_demand,lambdaij=1.2,a=0.3,p=1, r0=0.08, z=20, W=200*(10**4), q=5*(10**4), e=1.5*(10**4),mu=0.6, max_users=3, ni_min=10, ni_max=20, dmax=1.5, Dmin=0.5,penalty_F1=1000,penalty_F2=100,penalty_total=100):
    adder_text=""
    F1_=calculate_F1(charging_station_coordinates_list,demand_by_service_area,r0, z, W, q, e,mu, max_users,ni_min,ni_max,penalty_F1)
    F2_=calculate_F2(point_region_dictionary,charging_station_coordinates_list,EV_demand,lambdaij,a,p,dmax,penalty_F2)
    adder_text+=F1_[1]+F2_[1]
    F1=F1_[0]
    F2=F2_[0]
    other_costs=0
    final_objective=F1+F2
    for stn1 in EV_demand.keys():
        for stn2 in EV_demand.keys():
            if stn1!=stn2:
                Dist=np.sqrt(((stn1[0]-stn2[0])**2)+((stn1[1]-stn2[1])**2))
                #print(Dist)
                if (lambdaij*Dist)<Dmin:
                    final_objective+=((Dmin-(lambdaij*Dist))*penalty_total)
                    other_costs+=((Dmin-(lambdaij*Dist))*penalty_total)
    print("TOTAL OTHER COSTS=",other_costs)
    adder_text+="TOTAL OTHER COSTS="+str(other_costs)+"\n"
    print("F1 ",F1)
    adder_text+="F1 "+str(F1)+"\n"
    print("F2",F2)
    adder_text+="F2 "+str(F2)+"\n"
    print("FINAL OBJECTIVE",final_objective)
    adder_text+="FINAL OBJECTIVE "+str(final_objective)+"\n"
    

    return [adder_text,final_objective]

In [71]:
def mutate(population,threshold=0.5,min_X=0,min_Y=0,max_X=200,max_Y=300,capper=50) :
    for p in range(len(population)):
        for g in range(len(population[p])):
            x=population[p][g][0]
            y=population[p][g][1]
            x_old=x
            y_old=y
            
            if random.random()<0.95:
                if random.random()<0.5:
                    x+=random.randint(1,capper)
                    if random.random()<0.5:
                        y+=random.randint(1,capper)
                    else:
                        y-=random.randint(1,capper)
                else:
                    x-=random.randint(1,capper)
                    if random.random()<0.5:
                        y+=random.randint(1,capper)
                    else:
                        y-=random.randint(1,capper)
                
                if not(x>max_X or x<min_X or y<min_Y or y>max_Y):
                    #print("MUTATED SUCCESSFULLY")
                    population[p][g][0]=x
                    population[p][g][1]=y
    return population.copy()
                

In [72]:
def crossover(population):
    population_status=[]
    temp_pop=population.copy()
    child_pop=[]
    while population_status!=[True]*len(population):
        p1=random.choice(temp_pop).copy()
        temp_pop.remove(p1)
        p2=random.choice(temp_pop).copy()
        temp_pop.remove(p2)
        c1=p1.copy()
        c2=p2.copy()
        for i in range(len(c1)//2):
            temp=c1[i].copy()
            c1[i]=c2[i].copy()
            c2[i]=temp.copy()
        child_pop.append(c1.copy())
        child_pop.append(c2.copy())
        population_status.append(True)
        population_status.append(True)
    return child_pop


In [73]:
def selection(parent_population,child_population,population_score,child_population_score): # Modify this later on with a better selection strategy(like roulette wheel)
    new_population=[]
    for i in range(len(parent_population)):
        p1=random.randint(0,len(parent_population)-1)
        p2=random.randint(0,len(parent_population)-1)
        c1=random.randint(0,len(child_population)-1)
        c2=random.randint(0,len(child_population)-1)
        
        p1_score=population_score[p1]
        p2_score=population_score[p2]
        c1_score=child_population_score[c1]
        c2_score=child_population_score[c2]
        
        if p1_score<=p2_score and p1_score<=c1_score and p1_score<=c2_score:
            new_population.append(population[p1].copy())
        elif p2_score<=c1_score and p2_score<=c2_score:
            new_population.append(population[p2].copy())
        elif c1_score<=c2_score:
            new_population.append(child_population[c1].copy())
        else:
            new_population.append(child_population[c2].copy())
    return new_population.copy()


In [74]:
lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
def return_objective_value(pop,EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total):
    charging_station_coordinates=pop.copy()
    charging_station_coordinates_list=[]
    for point in charging_station_coordinates:
        charging_station_coordinates_list.append([float(point[0]),float(point[1])])
    points = np.array(charging_station_coordinates_list)
    vor = Voronoi(points.copy())
#     print("REGIONS \n",vor.regions)
#     print("VERTICES \n",vor.vertices)
#     print("POINT REGION \n",vor.point_region)
    region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
    #print(region_to_vertex_mapping)
    point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
#     print("POINT REGION DICTIONARY",point_region_dictionary[(135.0, 175.0)])
    charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
#     print("CHARGING STATION TO REGION NO\n",charging_station_to_region_no)
    charging_station_to_vertex=charging_station_vertex_mapper(vor)
#     print("CHARGING STATION TO VERTEX \n", charging_station_to_vertex)
    region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
#     print("REGION NO TO CHARGING_STATION",region_no_to_charging_station)
    total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
#    print("TOTAL DEMAND",total_demand)
#     xtemp=[]
#     ytemp=[]
#     text_=[]
#     for k,v in total_demand.items():
#         #print("K,V",k,v)
#         xtemp.append(k[0])
#         ytemp.append(k[1])
#         text_.append(v)
#     fig = plt.figure()
#     fig = voronoi_plot_2d(vor,point_size=30)
#     fig.set_figwidth(20)
#     fig.set_figheight(20)
#     plt.scatter(xcoords_centers,ycoords_centers,color='red',s=20)
#     plt.show()

#     plt.figure(figsize=(20,20))
#     plt.scatter(xtemp,ytemp)

#     for i in range(len(text_)):
#         plt.text(xtemp[i],ytemp[i],str(text_[i]),size=20)
    demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
    #print("DEMAND TO STATION",demand_point_to_charging_station_mapper)
    station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
    return obj
# import time
# no_of_members=10
# max_gen=100
# cur_gen=1
# nc=30
# population_tracker=[]
# import random
# population=[]
# for i in range(no_of_members):
#     temp=[]
#     for i in range(nc):
#         x=random.randint(1,max_X)
#         y=random.randint(1,max_Y)
#         temp.append([x,y])
#     population.append(temp)
# return_objective_value(population[0],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)

In [75]:
def return_best_solution(pop, pop_scores, best_solution_found,cur_gen):
    for i in range(len(pop_scores)):
        if best_solution_found[0]>pop_scores[i]:
            best_solution_found[0]=pop_scores[i]
            best_solution_found[1]=pop[i].copy()
            best_solution_found[2]=cur_gen
    return best_solution_found

### Initialization of parameters


In [76]:
import time
no_of_members=10
max_gen=1
cur_gen=1
nc=30
population_tracker=[]
import random
# population=[]
# for i in range(no_of_members):
#     temp=[]
#     for i in range(nc):
#         x=random.randint(1,max_X)
#         y=random.randint(1,max_Y)
#         temp.append([x,y])
#     population.append(temp)
# old_population=population.copy()


# definition of params

lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
global_tracker=[]
best_solution_found=[10**20,[],0]


In [77]:
import time

for nc in [30]:
    no_of_members=10
    max_gen=100  #----@#$%@&@#%#@$^#^#@-------------------------------------@#$%@#$@#%@#^@$^@#%@^@^26
    cur_gen=1
    best_solution_found=[10**20,[],0]
    population_tracker_text=""
    start=time.time()
    print("************************************ NC=",nc);
    population_tracker_text+="************************************ NC="+str(nc)+"\n "
    population=[]
    for i in range(no_of_members):
        temp=[]
        for i in range(nc):
            x=random.randint(1,max_X)
            y=random.randint(1,max_Y)
            temp.append([x,y])
        population.append(temp)
    population_tracker=[]
    population_tracker.append(population.copy())
    obj_tracker=[]

    while cur_gen<=max_gen:
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = ",cur_gen)
        population_tracker_text=population_tracker_text+"%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = "+str(cur_gen)+"\n "
        child_population=[]
        temp_population=crossover(population)
        child_population=mutate(temp_population,threshold=0.6,min_X=0,min_Y=0,max_X=max_X,max_Y=max_Y,capper=20)
        population_score={}
        child_population_score={}
        new_population_score={}
        for i in range(len(population)):
            print("PARENT POPULATION")
            population_tracker_text+="PARENT POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(population[i],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            population_score[i]=obj[1]
            population_tracker_text+=obj[0]
            obj_tracker.append((cur_gen,population[i],population_score[i],'P'))
        for j in range(len(child_population)):
            print("CHILD POPULATION")
            population_tracker_text+="CHILD POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(child_population[j],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            child_population_score[j]=obj[1]
            obj_tracker.append((cur_gen,child_population[j],obj,'C'))
            population_tracker_text+=obj[0]
        new_population=selection(population.copy(), child_population.copy(), population_score.copy(), child_population_score.copy()).copy()
        for k in range(len(new_population)):
            print("NEW POPULATION")
            population_tracker_text+="NEW POPULATION "+str(cur_gen)+"\n"
            obj=return_objective_value(new_population[k],EV_demand,lambdaij,a,p,r0,z,W,q,e,mu,max_users,ni_min,ni_max,dmax,Dmin,penalty_F1,penalty_F2,penalty_total)
            new_population_score[k]=obj[1]
            obj_tracker.append((cur_gen,new_population[k],obj,'N'))
            population_tracker_text+=obj[0]
        new_pop_tracker=return_best_solution(new_population.copy(),new_population_score,[10**20,[],0],cur_gen)
        population_tracker_text+="BEST SOLUTION FOUND IN THIS GENERATION"+str(new_pop_tracker)+"\n"
        best_solution_found=return_best_solution(new_population.copy(),new_population_score,best_solution_found,cur_gen)
        population=new_population.copy()
        cur_gen+=1
        population_tracker.append((cur_gen-1,new_population.copy(),new_population_score))
        #print("GENERATION=",cur_gen-1,"SCORE IS \n",new_population_score)
    
   
    population_tracker_text+="BEST SOLUTION FOUND UPTIL NOW"+str(best_solution_found)+"\n"
    end=time.time()
    global_tracker.append((nc,end-start,population_tracker.copy()))
    population_tracker_text+="################### TOTAL TIME TAKEN ################### = "+ str(end-start) +"\n"
    with open(str(nc)+" UPDATED GA EV LOG.txt", 'w') as f:
        f.write(population_tracker_text)


************************************ NC= 30
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION =  1
PARENT POPULATION
F1 OTHER COSTS= 27600000
F2 OTHER COSTS= 1028676.7420870548
TOTAL OTHER COSTS= 0
F1  594055500.0
F2 125076505.59587754
FINAL OBJECTIVE 719132005.5958775
PARENT POPULATION
F1 OTHER COSTS= 27500000
F2 OTHER COSTS= 1204033.9981791803
TOTAL OTHER COSTS= 0
F1  583755000.0
F2 145094677.9046865
FINAL OBJECTIVE 728849677.9046865
PARENT POPULATION
F1 OTHER COSTS= 23800000
F2 OTHER COSTS= 1034662.1783876042
TOTAL OTHER COSTS= 0
F1  557285000.0
F2 125837650.02581601
FINAL OBJECTIVE 683122650.025816
PARENT POPULATION
F1 OTHER COSTS= 25600000
F2 OTHER COSTS= 1034178.6590944335
TOTAL OTHER COSTS= 0
F1  550649750.0
F2 125413738.72145012
FINAL OBJECTIVE 676063488.7214501
PARENT POPULATION
F1 OTHER COSTS= 27600000
F2 OTHER COSTS= 1118354.2098962262
TOTAL OTHER COSTS= 0
F1  607706000.0
F2 135236763.15026525
FINAL OBJECTIVE 742942763.1502652
PARENT POPULATION
F1 OTHER COSTS= 26500000
F2 OTH

F1 OTHER COSTS= 24100000
F2 OTHER COSTS= 1191629.6132093666
TOTAL OTHER COSTS= 0
F1  527805750.0
F2 143550571.34671667
FINAL OBJECTIVE 671356321.3467166
CHILD POPULATION
F1 OTHER COSTS= 24000000
F2 OTHER COSTS= 1118204.7590733522
TOTAL OTHER COSTS= 0
F1  549164750.0
F2 135218203.91913375
FINAL OBJECTIVE 684382953.9191338
NEW POPULATION
F1 OTHER COSTS= 24100000
F2 OTHER COSTS= 1191629.6132093666
TOTAL OTHER COSTS= 0
F1  527805750.0
F2 143550571.34671667
FINAL OBJECTIVE 671356321.3467166
NEW POPULATION
F1 OTHER COSTS= 24000000
F2 OTHER COSTS= 1118204.7590733522
TOTAL OTHER COSTS= 0
F1  549164750.0
F2 135218203.91913375
FINAL OBJECTIVE 684382953.9191338
NEW POPULATION
F1 OTHER COSTS= 24000000
F2 OTHER COSTS= 1118204.7590733522
TOTAL OTHER COSTS= 0
F1  549164750.0
F2 135218203.91913375
FINAL OBJECTIVE 684382953.9191338
NEW POPULATION
F1 OTHER COSTS= 24000000
F2 OTHER COSTS= 1118204.7590733522
TOTAL OTHER COSTS= 0
F1  549164750.0
F2 135218203.91913375
FINAL OBJECTIVE 684382953.9191338
NEW P

F1 OTHER COSTS= 26000000
F2 OTHER COSTS= 1118685.9368772183
TOTAL OTHER COSTS= 0
F1  567707500.0
F2 135522278.3346427
FINAL OBJECTIVE 703229778.3346426
PARENT POPULATION
F1 OTHER COSTS= 26700000
F2 OTHER COSTS= 1169060.829666277
TOTAL OTHER COSTS= 0
F1  556177250.0
F2 141256096.05213517
FINAL OBJECTIVE 697433346.0521352
PARENT POPULATION
F1 OTHER COSTS= 24600000
F2 OTHER COSTS= 1047424.3044796085
TOTAL OTHER COSTS= 0
F1  555549250.0
F2 127087722.12489446
FINAL OBJECTIVE 682636972.1248945
PARENT POPULATION
F1 OTHER COSTS= 23900000
F2 OTHER COSTS= 1126895.9036344634
TOTAL OTHER COSTS= 0
F1  548093000.0
F2 136388058.25749078
FINAL OBJECTIVE 684481058.2574908
CHILD POPULATION
F1 OTHER COSTS= 26500000
F2 OTHER COSTS= 1072091.394459361
TOTAL OTHER COSTS= 0
F1  560657750.0
F2 130100595.72200385
FINAL OBJECTIVE 690758345.7220038
CHILD POPULATION
F1 OTHER COSTS= 24000000
F2 OTHER COSTS= 992444.7433068618
TOTAL OTHER COSTS= 0
F1  537664750.0
F2 120879135.79018238
FINAL OBJECTIVE 658543885.790182

TOTAL OTHER COSTS= 0
F1  523189250.0
F2 114865853.29449838
FINAL OBJECTIVE 638055103.2944984
NEW POPULATION
F1 OTHER COSTS= 20300000
F2 OTHER COSTS= 941518.881824704
TOTAL OTHER COSTS= 0
F1  523189250.0
F2 114865853.29449838
FINAL OBJECTIVE 638055103.2944984
NEW POPULATION
F1 OTHER COSTS= 22500000
F2 OTHER COSTS= 969550.2866623852
TOTAL OTHER COSTS= 0
F1  541506500.0
F2 117996692.19972628
FINAL OBJECTIVE 659503192.1997263
NEW POPULATION
F1 OTHER COSTS= 21700000
F2 OTHER COSTS= 960200.8439400821
TOTAL OTHER COSTS= 0
F1  530069000.0
F2 116964107.51932573
FINAL OBJECTIVE 647033107.5193257
NEW POPULATION
F1 OTHER COSTS= 20300000
F2 OTHER COSTS= 941518.881824704
TOTAL OTHER COSTS= 0
F1  523189250.0
F2 114865853.29449838
FINAL OBJECTIVE 638055103.2944984
NEW POPULATION
F1 OTHER COSTS= 22500000
F2 OTHER COSTS= 969550.2866623852
TOTAL OTHER COSTS= 0
F1  541506500.0
F2 117996692.19972628
FINAL OBJECTIVE 659503192.1997263
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION =  6
PARENT POPULATION
F1

TOTAL OTHER COSTS= 0
F1  590862250.0
F2 139977397.6468793
FINAL OBJECTIVE 730839647.6468793
CHILD POPULATION
F1 OTHER COSTS= 24800000
F2 OTHER COSTS= 1148452.8982356943
TOTAL OTHER COSTS= 0
F1  568922500.0
F2 138688114.7414573
FINAL OBJECTIVE 707610614.7414573
CHILD POPULATION
F1 OTHER COSTS= 25500000
F2 OTHER COSTS= 1083581.636382717
TOTAL OTHER COSTS= 0
F1  590127000.0
F2 131244671.02237444
FINAL OBJECTIVE 721371671.0223744
CHILD POPULATION
F1 OTHER COSTS= 27300000
F2 OTHER COSTS= 1154611.568962432
TOTAL OTHER COSTS= 0
F1  581260750.0
F2 139468858.57750976
FINAL OBJECTIVE 720729608.5775098
CHILD POPULATION
F1 OTHER COSTS= 26900000
F2 OTHER COSTS= 1074177.5542370318
TOTAL OTHER COSTS= 0
F1  599352750.0
F2 130140138.67360242
FINAL OBJECTIVE 729492888.6736025
CHILD POPULATION
F1 OTHER COSTS= 26100000
F2 OTHER COSTS= 1049528.1836507407
TOTAL OTHER COSTS= 0
F1  581515500.0
F2 127255693.83434269
FINAL OBJECTIVE 708771193.8343427
CHILD POPULATION
F1 OTHER COSTS= 25200000
F2 OTHER COSTS= 112

...

TOTAL OTHER COSTS= 0
F1  708921500.0
F2 125219538.55681206
FINAL OBJECTIVE 834141038.556812
PARENT POPULATION
F1 OTHER COSTS= 35700000
F2 OTHER COSTS= 1031508.7898529428
TOTAL OTHER COSTS= 0
F1  708921500.0
F2 125219538.55681206
FINAL OBJECTIVE 834141038.556812
PARENT POPULATION
F1 OTHER COSTS= 26800000
F2 OTHER COSTS= 989061.120577545
TOTAL OTHER COSTS= 0
F1  568438500.0
F2 120209464.07906131
FINAL OBJECTIVE 688647964.0790613
CHILD POPULATION
F1 OTHER COSTS= 27000000
F2 OTHER COSTS= 1039766.5554788879
TOTAL OTHER COSTS= 0
F1  560686250.0
F2 126109276.97492673
FINAL OBJECTIVE 686795526.9749267
CHILD POPULATION
F1 OTHER COSTS= 26400000
F2 OTHER COSTS= 971620.0559015449
TOTAL OTHER COSTS= 0
F1  566601000.0
F2 118235359.6811184
FINAL OBJECTIVE 684836359.6811184
CHILD POPULATION
F1 OTHER COSTS= 26700000
F2 OTHER COSTS= 1068044.6652831554
TOTAL OTHER COSTS= 0
F1  593489000.0
F2 129553743.80255535
FINAL OBJECTIVE 723042743.8025553
CHILD POPULATION
F1 OTHER COSTS= 26800000
F2 OTHER COSTS= 100

F1 OTHER COSTS= 26600000
F2 OTHER COSTS= 990137.6478435737
TOTAL OTHER COSTS= 0
F1  570107250.0
F2 120496426.44116221
FINAL OBJECTIVE 690603676.4411622
NEW POPULATION
F1 OTHER COSTS= 26000000
F2 OTHER COSTS= 1050076.0394496804
TOTAL OTHER COSTS= 0
F1  561721750.0
F2 127357946.8319404
FINAL OBJECTIVE 689079696.8319404
NEW POPULATION
F1 OTHER COSTS= 26600000
F2 OTHER COSTS= 1030592.5516021689
TOTAL OTHER COSTS= 0
F1  563759250.0
F2 125230591.24119143
FINAL OBJECTIVE 688989841.2411914
NEW POPULATION
F1 OTHER COSTS= 27300000
F2 OTHER COSTS= 1034763.518495516
TOTAL OTHER COSTS= 0
F1  571416750.0
F2 125627630.09439683
FINAL OBJECTIVE 697044380.0943968
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION =  96
PARENT POPULATION
F1 OTHER COSTS= 21500000
F2 OTHER COSTS= 935539.8886631598
TOTAL OTHER COSTS= 0
F1  532209250.0
F2 114126774.1908056
FINAL OBJECTIVE 646336024.1908056
PARENT POPULATION
F1 OTHER COSTS= 21200000
F2 OTHER COSTS= 964137.4472198732
TOTAL OTHER COSTS= 0
F1  539539500.0
F2 11748

F1 OTHER COSTS= 23200000
F2 OTHER COSTS= 992987.67322682
TOTAL OTHER COSTS= 0
F1  546329250.0
F2 120726394.34007196
FINAL OBJECTIVE 667055644.3400719
CHILD POPULATION
F1 OTHER COSTS= 24100000
F2 OTHER COSTS= 1105745.917064956
TOTAL OTHER COSTS= 0
F1  551260000.0
F2 133799942.01829764
FINAL OBJECTIVE 685059942.0182977
CHILD POPULATION
F1 OTHER COSTS= 23000000
F2 OTHER COSTS= 992774.5592490806
TOTAL OTHER COSTS= 0
F1  526964500.0
F2 120660720.2529563
FINAL OBJECTIVE 647625220.2529563
CHILD POPULATION
F1 OTHER COSTS= 23200000
F2 OTHER COSTS= 1004446.6158760219
TOTAL OTHER COSTS= 0
F1  536312750.0
F2 122119857.65039518
FINAL OBJECTIVE 658432607.6503952
CHILD POPULATION
F1 OTHER COSTS= 24200000
F2 OTHER COSTS= 1002749.8086833421
TOTAL OTHER COSTS= 0
F1  536116750.0
F2 122028528.73445235
FINAL OBJECTIVE 658145278.7344524
CHILD POPULATION
F1 OTHER COSTS= 23700000
F2 OTHER COSTS= 922374.3510530839
TOTAL OTHER COSTS= 0
F1  540901000.0
F2 112492293.4160086
FINAL OBJECTIVE 653393293.4160086
NEW P

TOTAL OTHER COSTS= 0
F1  540187750.0
F2 117340855.34415972
FINAL OBJECTIVE 657528605.3441597
PARENT POPULATION
F1 OTHER COSTS= 20800000
F2 OTHER COSTS= 934666.0145894252
TOTAL OTHER COSTS= 0
F1  519635500.0
F2 113900335.05389376
FINAL OBJECTIVE 633535835.0538938
PARENT POPULATION
F1 OTHER COSTS= 23800000
F2 OTHER COSTS= 980351.1923327402
TOTAL OTHER COSTS= 0
F1  541599000.0
F2 119247518.27995805
FINAL OBJECTIVE 660846518.279958
PARENT POPULATION
F1 OTHER COSTS= 23800000
F2 OTHER COSTS= 980351.1923327402
TOTAL OTHER COSTS= 0
F1  541599000.0
F2 119247518.27995805
FINAL OBJECTIVE 660846518.279958
PARENT POPULATION
F1 OTHER COSTS= 21900000
F2 OTHER COSTS= 963629.7800341595
TOTAL OTHER COSTS= 0
F1  540187750.0
F2 117340855.34415972
FINAL OBJECTIVE 657528605.3441597
PARENT POPULATION
F1 OTHER COSTS= 20800000
F2 OTHER COSTS= 934666.0145894252
TOTAL OTHER COSTS= 0
F1  519635500.0
F2 113900335.05389376
FINAL OBJECTIVE 633535835.0538938
PARENT POPULATION
F1 OTHER COSTS= 20800000
F2 OTHER COSTS= 

F1 OTHER COSTS= 20200000
F2 OTHER COSTS= 886080.5039304076
TOTAL OTHER COSTS= 0
F1  505362000.0
F2 108311607.3052315
FINAL OBJECTIVE 613673607.3052315
NEW POPULATION
F1 OTHER COSTS= 20200000
F2 OTHER COSTS= 937920.4941090603
TOTAL OTHER COSTS= 0
F1  512463250.0
F2 114362677.27923274
FINAL OBJECTIVE 626825927.2792327
NEW POPULATION
F1 OTHER COSTS= 20200000
F2 OTHER COSTS= 937920.4941090603
TOTAL OTHER COSTS= 0
F1  512463250.0
F2 114362677.27923274
FINAL OBJECTIVE 626825927.2792327
NEW POPULATION
F1 OTHER COSTS= 20200000
F2 OTHER COSTS= 886080.5039304076
TOTAL OTHER COSTS= 0
F1  505362000.0
F2 108311607.3052315
FINAL OBJECTIVE 613673607.3052315
NEW POPULATION
F1 OTHER COSTS= 20900000
F2 OTHER COSTS= 937470.2735968443
TOTAL OTHER COSTS= 0
F1  512519250.0
F2 114331090.7066829
FINAL OBJECTIVE 626850340.7066829
NEW POPULATION
F1 OTHER COSTS= 20900000
F2 OTHER COSTS= 937470.2735968443
TOTAL OTHER COSTS= 0
F1  512519250.0
F2 114331090.7066829
FINAL OBJECTIVE 626850340.7066829
NEW POPULATION
F1

##### Particle Swarm Optimization

In [30]:
def best_swarm_position_initializer(pso_population,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total):
    g=10**10
    best_swarm_value=10**10
    best_swarm_position=-1
    for keys,values in pso_population.items():
        charging_station_coordinates=values.copy()
        charging_station_coordinates_list=[]
        for point in charging_station_coordinates:
            charging_station_coordinates_list.append([float(point[0]),float(point[1])])
        points = np.array(charging_station_coordinates_list)
        vor = Voronoi(points.copy())
        region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
        point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
        charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
        charging_station_to_vertex=charging_station_vertex_mapper(vor)
        region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
        total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
        demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
        station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
        get_no_of_chargers(+total_demand ,mu, max_users)
        obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
        if obj[1]<best_swarm_value:
            best_swarm_position=values.copy()
            best_swarm_value=obj[1]
    return (best_swarm_position.copy(),best_swarm_value)

def get_objective_value(particle,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total):

    charging_station_coordinates=particle.copy()
    charging_station_coordinates_list=[]
    for point in charging_station_coordinates:
        charging_station_coordinates_list.append([float(point[0]),float(point[1])])
    points = np.array(charging_station_coordinates_list)
    vor = Voronoi(points.copy())
    region_to_vertex_mapping=region_to_vertex_mapper(vor.regions,vor.vertices)
    point_region_dictionary=get_point_belonging_dictionary(geometric_centers_list, region_to_vertex_mapping)
    charging_station_to_region_no=charging_station_to_region_no_mapper(vor)
    charging_station_to_vertex=charging_station_vertex_mapper(vor)
    region_no_to_charging_station=region_no_to_charging_station_mapper(vor, charging_station_to_region_no, point_region_dictionary, region_to_vertex_mapping)
    total_demand=get_demand_for_charging_station(point_region_dictionary,EV_demand,charging_station_coordinates_list)
    demand_point_to_charging_station_mapper=demand_point_to_station_mapper(point_region_dictionary)
    station_to_demand=station_to_demand_mapper(point_region_dictionary,charging_station_coordinates_list)
    get_no_of_chargers(total_demand ,mu, max_users)
    obj=calculate_final_objective(point_region_dictionary,charging_station_coordinates_list,total_demand, EV_demand,lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
    return obj
    

In [31]:
lambdaij=1.2
a=0.3
p=1
r0=0.08
z=20
W=200*(10**4)
q=5*(10**4)
e=1.5*(10**4)
mu=0.6
max_users=3
ni_min=10
ni_max=20
dmax=1.5*10 #1.5
Dmin=1*10 #0.5
penalty_F1=100000
penalty_F2=100
penalty_total=100
global_tracker=[]
PSO_best_solution_found=[10**20,[],0]
PSO_population_tracker_text=""

In [34]:

import time

for nc in [70]:
    PSO_best_solution_found=[10**20,[],0]
    print("************************************ NC=",nc);
    PSO_population_tracker_text+="************************************ NC="+str(nc)+"\n "
    PSO_population_tracker_text=""
    start=time.time()
    no_of_particles=20
    max_iter=2  #!#$@#%#@%&@$^@$&@$^@$^@W$%@%#$@$%^%@$%@################^@#%@#$@
    cur_iter=1
    pso_population_tracker=[]
    particle_best_position={}
    import random
    pso_position={}
    for i in range(no_of_particles):
        temp=[]
        for j in range(nc):
            x=random.randint(1,max_X)
            y=random.randint(1,max_Y)
            temp.append([x,y])
        pso_position[i]=temp.copy()
        #print(pso_position)
    for keys,values in pso_position.items():
        particle_best_position[keys]=get_objective_value(values.copy(),lambdaij,a,p, r0, z, W, q, e,mu, 
                                                                        max_users, ni_min, ni_max, dmax, 
                                                                        Dmin,penalty_F1,penalty_F2,penalty_total)[1]
        #print(particle_best_position[keys])
    best_swarm_position=best_swarm_position_initializer(pso_position,lambdaij,a,p,r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
    pso_velocity={}
    for i in range(no_of_particles):
        temp=[]
        for j in range(nc):
            x=random.randint(-(max_X-min_X),(max_X-min_X))
            y=random.randint(-(max_Y-min_Y),(max_Y-min_Y))
            temp.append([x,y])
        pso_velocity[i]=temp.copy()
    phig=random.uniform(1,3)
    phip=random.uniform(1,3)
    pso_population_tracker.append((pso_position.copy(),pso_velocity.copy()))  
    while cur_iter<=max_iter:
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = ",cur_iter)
        PSO_population_tracker_text+="%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% ITERATION = "+str(cur_iter)+"\n "
        PSO_score_list=[]
        for i in range(no_of_particles):
            for j in range(nc):
                rp=random.random()
                rg=random.random()
                w=random.random()
                pso_velocity[i][j][0]=int((w*pso_velocity[i][j][0])+(phip*rp*(particle_best_position[i][0][j][0]-pso_position[i][j][0]))+(phig*rg*(best_swarm_position[0][j][0]-pso_position[i][j][0])))
                pso_velocity[i][j][1]=int((w*pso_velocity[i][j][1])+(phip*rp*(particle_best_position[i][0][j][1]-pso_position[i][j][1]))+(phig*rg*(best_swarm_position[0][j][1]-pso_position[i][j][1])))
            new_coordinates=[]
            for j in range(nc):
                new_coordinates.append([pso_position[i][j][0]+pso_velocity[i][j][0],pso_position[i][j][1]+pso_velocity[i][j][1]])
            for j in range(nc):
                if not(new_coordinates[j][0]>=min_X and new_coordinates[j][0]<=max_X):
                    new_coordinates[j][0]=pso_position[i][j][0]
                if not(new_coordinates[j][1]>=min_Y and new_coordinates[j][1]<=max_Y):
                    new_coordinates[j][1]=pso_position[i][j][1]
            pso_position[i]=new_coordinates.copy()
            cur_value=get_objective_value(pso_position[i],lambdaij,a,p, r0, z, W, q, e,mu, max_users, ni_min, ni_max, dmax, Dmin,penalty_F1,penalty_F2,penalty_total)
            PSO_score_list.append(cur_value[1])
            PSO_population_tracker_text+=str(cur_value[1])+"\n"
            PSO_population_tracker_text+=str(cur_value[0])+"\n"
            if cur_value[1]<particle_best_position[i][1]:
                particle_best_position[i]=(pso_position[i].copy(),cur_value[1])
            if cur_value[1]<best_swarm_position[1]:
                best_swarm_position=(pso_position[i].copy(),cur_value[1])
                PSO_best_solution_found=[cur_value[1],pso_position[i].copy(),cur_iter]
        cur_iter+=1            
        #print(best_swarm_position,cur_iter)
        pso_population_tracker.append((pso_position.copy(),pso_velocity.copy())) 
        new_pop_PSO_tracker=return_best_solution(list(pso_position.values()).copy(),PSO_score_list.copy(),[10**20,[],0],cur_iter)
        PSO_population_tracker_text+="BEST SOLUTION FOUND IN THIS GENERATION "+str(new_pop_PSO_tracker)+"\n"
    PSO_population_tracker_text+="BEST SOLUTION FOUND UPTIL NOW "+str(PSO_best_solution_found)+"\n"
    end=time.time()
    PSO_population_tracker_text+="################### TOTAL TIME TAKEN ################### = "+ str(end-start) +"\n"
    with open(str(nc)+" UPDATED PSO EV LOG.txt", 'w') as f:
        f.write(PSO_population_tracker_text)


************************************ NC= 70
F1 OTHER COSTS= 15000000
F2 OTHER COSTS= 430051.57586307544
TOTAL OTHER COSTS= 0
F1  399876250.0
F2 55151560.147578314
FINAL OBJECTIVE 455027810.1475783
F1 OTHER COSTS= 14300000
F2 OTHER COSTS= 405236.20131894236
TOTAL OTHER COSTS= 0
F1  394392250.0
F2 52284930.94778643
FINAL OBJECTIVE 446677180.94778645
F1 OTHER COSTS= 14000000
F2 OTHER COSTS= 412328.3770433368
TOTAL OTHER COSTS= 0
F1  395880500.0
F2 53188050.70776745
FINAL OBJECTIVE 449068550.7077674
F1 OTHER COSTS= 16900000
F2 OTHER COSTS= 514893.6751482412
TOTAL OTHER COSTS= 0
F1  408147250.0
F2 64939187.84542526
FINAL OBJECTIVE 473086437.84542525
F1 OTHER COSTS= 16700000
F2 OTHER COSTS= 404565.46574731026
TOTAL OTHER COSTS= 0
F1  408200250.0
F2 52156547.64459352
FINAL OBJECTIVE 460356797.64459354
F1 OTHER COSTS= 18000000
F2 OTHER COSTS= 467329.55155228707
TOTAL OTHER COSTS= 0
F1  412036000.0
F2 59402444.34910037
FINAL OBJECTIVE 471438444.34910035
F1 OTHER COSTS= 13700000
F2 OTHER COSTS= 

TypeError: bad operand type for unary +: 'dict'

In [ ]:
# End of Project File 
# Ram Ram